# FEMA DATA

This dataset is used to determine the houses that was damaged by flood and the amount of claim that was paid by FEMA to damaged house. There are additional information for each house including lat, long, year the house was built, number of stories, city, zip code, etc. 

The scope if to find another dataset from NASA that includes percipitation. Then combine the 2 dataset and train the model to predict the expected "claim" based on "percipitation". Additional data will be used to better train the model including location, year that the building was built, etc. 

## A. READ FEMA DATA from CSV FILE

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf
from pathlib import Path

In [10]:
# Import our input dataset
FEMA_df = pd.read_csv('../UCB_Final_Project/Data/openfema_claims20190331.csv')
FEMA_df.head()

//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,11,15,22,23,31,32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,agriculturestructureindicator,asofdate,basefloodelevation,basementenclosurecrawlspacetype,reportedcity,condominiumindicator,policycount,countycode,crsdiscount,dateofloss,...,amountpaidonincreasedcostofcomplianceclaim,postfirmconstructionindicator,ratemethod,smallbusinessindicatorbuilding,state,totalbuildinginsurancecoverage,totalcontentsinsurancecoverage,yearofloss,reportedzipcode,primaryresidence
0,NaN,2019-03-31,NaN,0.0,OCEANSIDE,N,1.0,6073.0,0.00,1998-02-07,...,NaN,N,7,NaN,CA,200000.0,50000.0,1998,92056,NaN
1,NaN,2019-03-31,NaN,0.0,NEW ORLEANS,N,1.0,22071.0,0.00,2005-08-29,...,NaN,N,7,NaN,LA,100000.0,40000.0,2005,70131,Y
2,NaN,2019-03-31,NaN,0.0,NAVARRE,N,1.0,12113.0,0.05,1998-09-28,...,0.0,N,1,NaN,FL,100000.0,50000.0,1998,32566,NaN
3,NaN,2019-03-31,NaN,1.0,BEAUFORT,N,1.0,45013.0,0.00,1994-10-07,...,NaN,N,7,NaN,SC,100000.0,25000.0,1994,29902,NaN
4,NaN,2019-03-31,NaN,0.0,MELBOURNE,N,1.0,12009.0,0.00,1996-03-11,...,NaN,Y,7,NaN,FL,100000.0,25000.0,1996,32940,NaN


## B. Remove Unnecessary Columns From FEMA Dataset

### B.1. Remove columns that are obviously not adding any value

In [11]:
FEMA_Clean_df=FEMA_df.drop(columns=["policycount","countycode","crsdiscount","condominiumindicator","agriculturestructureindicator",
                                    "basementenclosurecrawlspacetype",'asofdate',"amountpaidoncontentsclaim","obstructiontype",
                      'basefloodelevation',"elevationcertificateindicator","elevatedbuildingindicator","censustract",
                      "houseworship","locationofcontents","lowestadjacentgrade","postfirmconstructionindicator","yearofloss",
                      "lowestfloorelevation","nonprofitindicator","originalnbdate","amountpaidonincreasedcostofcomplianceclaim",
                      "smallbusinessindicatorbuilding","primaryresidence","ratemethod","totalbuildinginsurancecoverage","totalcontentsinsurancecoverage"])
FEMA_Clean_df.head()

,reportedcity,dateofloss,elevationdifference,floodzone,latitude,longitude,numberoffloorsintheinsuredbuilding,occupancytype,originalconstructiondate,amountpaidonbuildingclaim,state,reportedzipcode
0,OCEANSIDE,1998-02-07,999.0,X,33.2,-117.3,4.0,1.0,1963-01-01,0.00,CA,92056
1,NEW ORLEANS,2005-08-29,999.0,X,29.9,-90.0,2.0,1.0,1967-07-01,0.00,LA,70131
2,NAVARRE,1998-09-28,999.0,X,30.4,-86.9,1.0,1.0,1972-01-01,8813.21,FL,32566
3,BEAUFORT,1994-10-07,999.0,X,32.4,-80.7,2.0,1.0,1960-01-01,2906.00,SC,29902
4,MELBOURNE,1996-03-11,999.0,X,28.3,-80.7,1.0,1.0,1988-01-01,3875.53,FL,32940


In [12]:
FEMA_Clean_df.shape

(2418007, 12)

In [13]:
FEMA_Clean_df.dtypes

reportedcity                           object
dateofloss                             object
elevationdifference                   float64
floodzone                              object
latitude                              float64
longitude                             float64
numberoffloorsintheinsuredbuilding    float64
occupancytype                         float64
originalconstructiondate               object
amountpaidonbuildingclaim             float64
state                                  object
reportedzipcode                        object
dtype: object

### B.2. Remove Columnms that has many empty cells and check to see remaining columns dont have many empty cells

In [14]:
#Remove all Data that don’t have an City defined.
FEMA_Clean1_df=FEMA_Clean_df[FEMA_Clean_df["reportedcity"] != "N/A"]
FEMA_Clean1_df.dropna(subset=['reportedcity'], how='all', inplace=True)
print(FEMA_Clean1_df.shape)

FEMA_Clean2_df=FEMA_Clean1_df[FEMA_Clean1_df["amountpaidonbuildingclaim"] != "N/A"]
FEMA_Clean2_df.dropna(subset=['amountpaidonbuildingclaim'], how='all', inplace=True)
print(FEMA_Clean2_df.shape)

FEMA_Clean3_df=FEMA_Clean2_df[FEMA_Clean2_df["originalconstructiondate"] != "N/A"]
FEMA_Clean3_df.dropna(subset=['originalconstructiondate'], how='all', inplace=True)
print(FEMA_Clean3_df.shape)

FEMA_Clean4_df=FEMA_Clean3_df[FEMA_Clean3_df["floodzone"] != "N/A"]
FEMA_Clean4_df.dropna(subset=['floodzone'], how='all', inplace=True)
print(FEMA_Clean4_df.shape)

FEMA_Clean5_df=FEMA_Clean4_df[FEMA_Clean4_df["occupancytype"] != "N/A"]
FEMA_Clean5_df.dropna(subset=['occupancytype'], how='all', inplace=True)
print(FEMA_Clean5_df.shape)

FEMA_Clean6_df=FEMA_Clean5_df[FEMA_Clean5_df["numberoffloorsintheinsuredbuilding"] != "N/A"]
FEMA_Clean6_df.dropna(subset=['numberoffloorsintheinsuredbuilding'], how='all', inplace=True)
print(FEMA_Clean6_df.shape)

FEMA_Clean7_df=FEMA_Clean6_df[FEMA_Clean6_df["latitude"] != "N/A"]
FEMA_Clean7_df.dropna(subset=['latitude'], how='all', inplace=True)
print(FEMA_Clean7_df.shape)

FEMA_Clean8_df=FEMA_Clean7_df[FEMA_Clean7_df["occupancytype"] != "N/A"]
FEMA_Clean8_df.dropna(subset=['longitude'], how='all', inplace=True)
print(FEMA_Clean8_df.shape)

FEMA_Clean9_df=FEMA_Clean8_df[FEMA_Clean8_df["dateofloss"] != "N/A"]
FEMA_Clean9_df.dropna(subset=['dateofloss'], how='all', inplace=True)
print(FEMA_Clean9_df.shape)

FEMA_Clean10_df=FEMA_Clean9_df[FEMA_Clean9_df["reportedzipcode"] != "N/A"]
FEMA_Clean10_df.dropna(subset=['reportedzipcode'], how='all', inplace=True)
print(FEMA_Clean10_df.shape)

FEMA_Clean11_df=FEMA_Clean10_df[FEMA_Clean10_df["reportedzipcode"] != "N/A"]
FEMA_Clean11_df.dropna(subset=['reportedzipcode'], how='all', inplace=True)
print(FEMA_Clean11_df.shape)

FEMA_Clean12_df=FEMA_Clean11_df[FEMA_Clean11_df["elevationdifference"] != "N/A"]
FEMA_Clean12_df.dropna(subset=['elevationdifference'], how='all', inplace=True)
print(FEMA_Clean12_df.shape)

FEMA_Clean13_df=FEMA_Clean12_df[FEMA_Clean12_df["state"] != "N/A"]
FEMA_Clean13_df.dropna(subset=['state'], how='all', inplace=True)
print(FEMA_Clean13_df.shape)

(2413501, 12)


//anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


(2340566, 12)
(2006932, 12)
(1986720, 12)
(1986701, 12)
(1986699, 12)
(1972517, 12)
(1972517, 12)
(1972517, 12)
(1972396, 12)
(1972396, 12)
(1972396, 12)
(1972396, 12)


### B.3. Check to make sure there is no empty cells in the dataframe

In [15]:
# Drop rows with any empty cells, check
FEMA_Clean14_df=FEMA_Clean13_df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
print(FEMA_Clean14_df.shape)
FEMA_Clean14_df

(1972396, 12)


,reportedcity,dateofloss,elevationdifference,floodzone,latitude,longitude,numberoffloorsintheinsuredbuilding,occupancytype,originalconstructiondate,amountpaidonbuildingclaim,state,reportedzipcode
0,OCEANSIDE,1998-02-07,999.0,X,33.2,-117.3,4.0,1.0,1963-01-01,0.00,CA,92056
1,NEW ORLEANS,2005-08-29,999.0,X,29.9,-90.0,2.0,1.0,1967-07-01,0.00,LA,70131
2,NAVARRE,1998-09-28,999.0,X,30.4,-86.9,1.0,1.0,1972-01-01,8813.21,FL,32566
3,BEAUFORT,1994-10-07,999.0,X,32.4,-80.7,2.0,1.0,1960-01-01,2906.00,SC,29902
4,MELBOURNE,1996-03-11,999.0,X,28.3,-80.7,1.0,1.0,1988-01-01,3875.53,FL,32940
...,...,...,...,...,...,...,...,...,...,...,...,...
2418002,MIAMI,2005-08-26,1.0,AHB,25.6,-80.4,1.0,1.0,1985-01-01,4390.96,FL,33177
2418003,MIAMI,2005-08-25,1.0,AHB,25.6,-80.4,1.0,1.0,1990-01-01,3541.59,FL,33186
2418004,PINECREST,2005-06-20,-1.0,AE,25.7,-80.3,1.0,1.0,1959-01-01,8335.35,FL,33156
2418005,PINECREST,2005-08-25,999.0,AE,25.7,-80.3,1.0,1.0,1959-01-01,2396.44,FL,33156


## C. Check FEMA DataFrame Data-types and convert dates to DateTime

In [16]:
FEMA_Clean14_df.dtypes

reportedcity                           object
dateofloss                             object
elevationdifference                   float64
floodzone                              object
latitude                              float64
longitude                             float64
numberoffloorsintheinsuredbuilding    float64
occupancytype                         float64
originalconstructiondate               object
amountpaidonbuildingclaim             float64
state                                  object
reportedzipcode                        object
dtype: object

In [17]:
#convert dateofloss type to datetime
FEMA_Clean14_df['dateofloss'] =  pd.to_datetime(FEMA_Clean14_df['dateofloss'], format='%Y-%m-%d')
FEMA_Clean14_df.dtypes

reportedcity                                  object
dateofloss                            datetime64[ns]
elevationdifference                          float64
floodzone                                     object
latitude                                     float64
longitude                                    float64
numberoffloorsintheinsuredbuilding           float64
occupancytype                                float64
originalconstructiondate                      object
amountpaidonbuildingclaim                    float64
state                                         object
reportedzipcode                               object
dtype: object

In [18]:
#Identify which row makes error on originalconstructiondate and doesnt let to change the type to date
FEMA_Clean14_df['originalconstructiondate'] =  pd.to_datetime(FEMA_Clean14_df['originalconstructiondate'], errors='coerce')
FEMA_Clean14_df.sort_values('originalconstructiondate')

,reportedcity,dateofloss,elevationdifference,floodzone,latitude,longitude,numberoffloorsintheinsuredbuilding,occupancytype,originalconstructiondate,amountpaidonbuildingclaim,state,reportedzipcode
178407,CHARLESTON,2015-10-05,-9.0,AE,32.8,-79.9,3.0,1.0,1860-01-01,20346.88,SC,29401
1229487,CHARLESTON,2016-10-08,999.0,AE,32.8,-79.9,3.0,6.0,1876-07-01,36988.33,SC,29401
1706519,SAVANNAH,1999-09-15,999.0,X,32.1,-81.1,3.0,1.0,1880-01-01,2387.46,GA,31401
1710556,MANDEVILLE,2002-09-26,999.0,AE,30.4,-90.1,2.0,1.0,1888-12-31,2803.92,LA,70448
193765,WAHPETON,1997-04-10,999.0,X,46.3,-96.6,1.0,1.0,1891-07-01,1642.58,ND,58075
...,...,...,...,...,...,...,...,...,...,...,...,...
988696,CRUMP,2019-02-20,1.0,AE,35.2,-88.3,1.0,1.0,2018-10-01,0.00,TN,38327
2335163,MORSE BLUFF,2019-03-18,2.0,AE,41.4,-96.7,2.0,1.0,2018-11-07,0.00,NE,68648
1054481,SUN VALLEY,2019-02-02,999.0,X,34.2,-118.4,1.0,1.0,2018-11-16,0.00,CA,91352
619900,CEDAR ISLAND,1999-08-30,999.0,A07,35.0,-76.3,1.0,1.0,2019-01-01,3277.32,NC,28520


In [19]:
FEMA_Clean14_df.dtypes

reportedcity                                  object
dateofloss                            datetime64[ns]
elevationdifference                          float64
floodzone                                     object
latitude                                     float64
longitude                                    float64
numberoffloorsintheinsuredbuilding           float64
occupancytype                                float64
originalconstructiondate              datetime64[ns]
amountpaidonbuildingclaim                    float64
state                                         object
reportedzipcode                               object
dtype: object

## D. Due to processing time, limit the Data to only 2 years (2014-2015) 

In [20]:
FEMA_Clean15_df=FEMA_Clean14_df[FEMA_Clean14_df["dateofloss"] > "2013-12-31"]
FEMA_Clean15_df

,reportedcity,dateofloss,elevationdifference,floodzone,latitude,longitude,numberoffloorsintheinsuredbuilding,occupancytype,originalconstructiondate,amountpaidonbuildingclaim,state,reportedzipcode
6,HOUSTON,2017-08-27,999.0,X,29.7,-95.5,1.0,1.0,1974-01-01,195857.43,TX,77096
48,NO CHATHAM,2015-02-21,999.0,AE,41.7,-70.0,3.0,1.0,1955-01-01,2618.18,MA,2650
83,SCITUATE,2018-03-02,999.0,X,42.2,-70.7,3.0,1.0,1981-07-01,3201.31,MA,2066
84,SCITUATE,2018-06-09,999.0,X,42.2,-70.7,3.0,1.0,1981-07-01,0.00,MA,2066
110,NEW BRAUNFELS,2015-10-30,2.0,AE,29.7,-98.1,3.0,1.0,2003-12-02,3309.59,TX,78130
...,...,...,...,...,...,...,...,...,...,...,...,...
2417992,NEW BRAUNFELS,2015-10-30,999.0,X,29.7,-98.2,1.0,1.0,1974-01-01,13929.98,TX,78130
2417993,KINGWOOD,2017-08-27,3.0,AE,30.1,-95.1,1.0,1.0,2002-01-01,250000.00,TX,77345
2417994,HOUSTON,2017-08-29,999.0,X,29.9,-95.3,1.0,1.0,1971-01-01,101508.49,TX,77039
2417996,HOUSTON,2017-08-26,999.0,X,29.8,-95.2,1.0,1.0,1952-06-01,76068.15,TX,77015


In [21]:
FEMA_Clean15_df=FEMA_Clean15_df[FEMA_Clean15_df["dateofloss"] < "2016-01-01"]
FEMA_Clean15_df

,reportedcity,dateofloss,elevationdifference,floodzone,latitude,longitude,numberoffloorsintheinsuredbuilding,occupancytype,originalconstructiondate,amountpaidonbuildingclaim,state,reportedzipcode
48,NO CHATHAM,2015-02-21,999.0,AE,41.7,-70.0,3.0,1.0,1955-01-01,2618.18,MA,2650
110,NEW BRAUNFELS,2015-10-30,2.0,AE,29.7,-98.1,3.0,1.0,2003-12-02,3309.59,TX,78130
152,PENSACOLA,2014-04-30,2.0,AE,30.3,-87.5,2.0,1.0,1982-07-01,0.00,FL,32507
299,EDISTO BEACH,2015-10-03,5.0,AE,32.5,-80.3,1.0,1.0,1980-01-01,0.00,SC,29438
369,PENSACOLA,2014-04-29,999.0,X,30.4,-87.3,2.0,1.0,1996-01-01,26119.37,FL,32506
...,...,...,...,...,...,...,...,...,...,...,...,...
2417986,HOUSTON,2015-05-25,999.0,X,29.7,-95.6,1.0,1.0,1967-01-15,7901.12,TX,77099
2417989,HOUSTON,2014-05-28,999.0,X,29.8,-95.5,1.0,1.0,1972-06-01,2259.89,TX,77080
2417990,HOUSTON,2015-05-25,999.0,X,29.8,-95.5,1.0,1.0,1972-06-01,1638.83,TX,77080
2417992,NEW BRAUNFELS,2015-10-30,999.0,X,29.7,-98.2,1.0,1.0,1974-01-01,13929.98,TX,78130


In [22]:
FEMA_Clean15_df.sort_values('dateofloss')

,reportedcity,dateofloss,elevationdifference,floodzone,latitude,longitude,numberoffloorsintheinsuredbuilding,occupancytype,originalconstructiondate,amountpaidonbuildingclaim,state,reportedzipcode
260533,MIAMI,2014-01-01,0.0,AHB,25.8,-80.3,1.0,1.0,1977-01-01,0.00,FL,33174
1556515,BOYNTON BEACH,2014-01-01,999.0,B,26.5,-80.2,1.0,1.0,1997-01-01,0.00,FL,33437
867379,SEBASTIAN,2014-01-01,999.0,X,27.8,-80.5,1.0,1.0,1981-01-01,0.00,FL,32958
42398,HAYES,2014-01-01,1.0,AE,37.3,-76.5,2.0,1.0,1985-01-01,0.00,VA,23072
72967,JOHNS ISLAND,2014-01-01,999.0,X,32.7,-80.1,1.0,1.0,2006-01-01,0.00,SC,29455
...,...,...,...,...,...,...,...,...,...,...,...,...
632950,JEFFERSON CITY,2015-12-31,999.0,X,38.6,-92.1,4.0,1.0,1970-05-15,0.00,MO,65101
2397951,PRTG DE SIOUX,2015-12-31,999.0,AE,38.9,-90.3,2.0,1.0,1950-05-01,3182.17,MO,63373
301569,KIRKWOOD,2015-12-31,999.0,A,38.6,-90.4,1.0,4.0,1966-01-01,12982.14,MO,63122
302135,SAINT LOUIS,2015-12-31,999.0,AE,38.7,-90.3,2.0,1.0,1955-01-01,0.00,MO,63136


### D.1. Remove the row in originalconstructiondate that causes error, this does not happen for years 2014 to 2015


In [23]:
#Remove the row in originalconstructiondate that causes error, this does not happen for years 2014 to 2015
# FEMA_Clean15_df=FEMA_Clean14_df.drop(549916)

In [24]:
#convert originalconstructiondate type to datetime
FEMA_Clean15_df['originalconstructiondate'] =  pd.to_datetime(FEMA_Clean15_df['originalconstructiondate'], format='%Y-%m-%d', errors='ignore')
FEMA_Clean15_df.dtypes

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


reportedcity                                  object
dateofloss                            datetime64[ns]
elevationdifference                          float64
floodzone                                     object
latitude                                     float64
longitude                                    float64
numberoffloorsintheinsuredbuilding           float64
occupancytype                                float64
originalconstructiondate              datetime64[ns]
amountpaidonbuildingclaim                    float64
state                                         object
reportedzipcode                               object
dtype: object

## E. Convert FEMA DataFram to GeoDataFrame
This is done to create a "point" for each house that was damaged. Later we can use this point to find the closest weather station to damaged house.

In [25]:
from geopandas import GeoDataFrame
from shapely.geometry import Point
import pandas as pd
import geopandas
import matplotlib.pyplot as plt

In [26]:
gdf_FEMA = geopandas.GeoDataFrame(
    FEMA_Clean15_df, geometry=geopandas.points_from_xy(FEMA_Clean15_df.longitude, FEMA_Clean15_df.latitude))

In [27]:
gdf_FEMA

,reportedcity,dateofloss,elevationdifference,floodzone,latitude,longitude,numberoffloorsintheinsuredbuilding,occupancytype,originalconstructiondate,amountpaidonbuildingclaim,state,reportedzipcode,geometry
48,NO CHATHAM,2015-02-21,999.0,AE,41.7,-70.0,3.0,1.0,1955-01-01,2618.18,MA,2650,POINT (-70.00000 41.70000)
110,NEW BRAUNFELS,2015-10-30,2.0,AE,29.7,-98.1,3.0,1.0,2003-12-02,3309.59,TX,78130,POINT (-98.10000 29.70000)
152,PENSACOLA,2014-04-30,2.0,AE,30.3,-87.5,2.0,1.0,1982-07-01,0.00,FL,32507,POINT (-87.50000 30.30000)
299,EDISTO BEACH,2015-10-03,5.0,AE,32.5,-80.3,1.0,1.0,1980-01-01,0.00,SC,29438,POINT (-80.30000 32.50000)
369,PENSACOLA,2014-04-29,999.0,X,30.4,-87.3,2.0,1.0,1996-01-01,26119.37,FL,32506,POINT (-87.30000 30.40000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2417986,HOUSTON,2015-05-25,999.0,X,29.7,-95.6,1.0,1.0,1967-01-15,7901.12,TX,77099,POINT (-95.60000 29.70000)
2417989,HOUSTON,2014-05-28,999.0,X,29.8,-95.5,1.0,1.0,1972-06-01,2259.89,TX,77080,POINT (-95.50000 29.80000)
2417990,HOUSTON,2015-05-25,999.0,X,29.8,-95.5,1.0,1.0,1972-06-01,1638.83,TX,77080,POINT (-95.50000 29.80000)
2417992,NEW BRAUNFELS,2015-10-30,999.0,X,29.7,-98.2,1.0,1.0,1974-01-01,13929.98,TX,78130,POINT (-98.20000 29.70000)


# NASA Data

In [40]:
from pylab import *
import netCDF4
import xarray as xr
import numpy as np

ModuleNotFoundError: No module named 'netCDF4'

## A. Read Data and Create a 1 combined DataFrame (Limit the data to only 2 years due to long processing time)

The NASA 2 years will match the FEMA 2 years (2014-2015)

### A.1. Read NASA datat in .nc4 format

In [31]:
# ds_2010 = xr.open_dataset('NASA/data/daymet_v3_stnsxval_prcp_2010.nc4',drop_variables=["stns","station_id","station_name","stnz","time_bnds", "stnx","stny"])
# ds_2011 = xr.open_dataset('NASA/data/daymet_v3_stnsxval_prcp_2011.nc4',drop_variables=["stns","station_id","station_name","stnz","time_bnds", "stnx","stny"])
# ds_2012 = xr.open_dataset('NASA/data/daymet_v3_stnsxval_prcp_2012.nc4',drop_variables=["stns","station_id","station_name","stnz","time_bnds", "stnx","stny"])
# ds_2013 = xr.open_dataset('NASA/data/daymet_v3_stnsxval_prcp_2013.nc4',drop_variables=["stns","station_id","station_name","stnz","time_bnds", "stnx","stny"])
ds_2014 = xr.open_dataset('NASA/data/daymet_v3_stnsxval_prcp_2014.nc4',drop_variables=["stns","station_id","station_name","stnz","time_bnds", "stnx","stny"])
ds_2015 = xr.open_dataset('NASA/data/daymet_v3_stnsxval_prcp_2015.nc4',drop_variables=["stns","station_id","station_name","stnz","time_bnds", "stnx","stny"])
ds_2015

NameError: name 'xr' is not defined

### A.2. Save data in a DataFrame

In [ ]:
# ds_2010 = ds_2010.to_dataframe()
# ds_2011 = ds_2011.to_dataframe()
# ds_2012 = ds_2012.to_dataframe()
# ds_2013 = ds_2013.to_dataframe()
ds_2014 = ds_2014.to_dataframe()
ds_2015 = ds_2015.to_dataframe()
ds_2015

### A.3. Reset index

In [ ]:
# ds_2010=ds_2010.reset_index()
# ds_2011=ds_2011.reset_index()
# ds_2012=ds_2012.reset_index()
# ds_2013=ds_2013.reset_index()
ds_2014=ds_2014.reset_index()
ds_2015=ds_2015.reset_index()
ds_2015

### A.4. Combine the NASA DataFrames

In [ ]:
NASA_Comb = pd.concat([
ds_2014,
ds_2015], ignore_index=True, sort=False)

## B. Convert timestep column to Date Column

This is necessary, because the FEMA dataset has only the "Data" column, which will be used to merge the data.

In [ ]:
NASA_Comb['time'] = pd.to_datetime(NASA_Comb['time'])
NASA_Comb['new_date_column'] = NASA_Comb['time'].dt.date
NASA_Comb['new_date_column'] =  pd.to_datetime(NASA_Comb['new_date_column'], format='%Y-%m-%d')


In [ ]:
NASA_Comb

In [ ]:
NASA_Comb.dtypes

## C. Convert DataFrame to GeoDataFrame

In [ ]:
gdf_NASA = geopandas.GeoDataFrame(
    NASA_Comb, geometry=geopandas.points_from_xy(NASA_Comb.stn_lon, NASA_Comb.stn_lat))

In [ ]:
# gdf_NASA.to_csv('gdf_NASA.csv')

In [ ]:
# gdf_NASA=pd.read_csv('gdf_NASA.csv')
# gdf_NASA.head(15)

## D. Convert all nan Data on "obs" Column to "0"

obs represents the amout the rainfall that the weather station has recorded on a specific date
Many obs data was not recorded and the cell was empty. Therefore it was assumed that there was no rain on that specific date. We do underestand that this may cause some inaccuracy. 

In [ ]:
gdf_NASA['obs'] = gdf_NASA['obs'].fillna(0.0)
gdf_NASA

# Combine FEMA and NASA Datasets

## A. Create a Function to find the closest distance between the points between FEMA and NASA Data, and Merge the 2 datasets

The function creates a new column called "dist", which represents the disctance between the 2 "Points" from each dataset.

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd

from scipy.spatial import cKDTree
from shapely.geometry import Point

# gpd1 = gpd.GeoDataFrame([['John', 1, Point(1, 1)], ['Smith', 1, Point(2, 2)],
#                          ['Soap', 1, Point(0, 2)]],
#                         columns=['Name', 'ID', 'geometry'])
# gpd2 = gpd.GeoDataFrame([['Work', Point(0, 1.1)], ['Shops', Point(2.5, 2)],
#                          ['Home', Point(1, 1.1)]],
#                         columns=['Place', 'geometry'])

def ckdnearest(gdA, gdB):
    nA = np.array(list(zip(gdA.geometry.x, gdA.geometry.y)) )
    nB = np.array(list(zip(gdB.geometry.x, gdB.geometry.y)) )
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdf = pd.concat(
        [gdA.reset_index(drop=True), gdB.loc[idx, gdB.columns != 'geometry'].reset_index(drop=True),
         pd.Series(dist, name='dist')], axis=1)
    return gdf

ckdnearest(gpd1, gpd2)

In [ ]:
#Run the function
gdf_FEMA_plus_station_id=ckdnearest(gdf_FEMA, gdf_NASA) #gdf_NASA[:10000])

## B. Merge FEMA and NASA datasets

The fist merged dataset is created based on the closest distance between points. 
From this merge we can find the closest wether station to each damaged hourse. See column "stns"
But the function doesn not look at matching the date from FEMA "dateofloss" and the date from NASA "new_date_column"

Considering we were able to match the weather station "stns" from NASA to FEMA database, we need to do the following:

### B.1. Remove Columns from "first merged" dataset that does not have correct value "time", "obs", "pred", "stn_lat", "stn_lon", "new_date_column", "dist", "geometry". 

### B.2. Keep Station column "stns" in the "first merged" dataset 

### B.3 Calculate the comulative amount of rain in the last 2 days and save it in a new column

### B.4. Merge NASA dataset again with "first merged" dataset based on the Station and Date

In [ ]:
gdf_FEMA_plus_station_id.head()

### B.1. Remove Columns from "first merged" dataset that does not have correct value.
### B.2. Keep Station column "stns" in the "first merged" dataset 

In [ ]:
gdf_FEMA_plus_station_id=gdf_FEMA_plus_station_id.drop(columns=["time","obs","pred","stn_lat","stn_lon","new_date_column","dist","geometry"])

In [ ]:
gdf_FEMA_plus_station_id

### B.3 Calculate the comulative amount of rain in the last 2 days and save it in a new column

In [ ]:
gdf_NASA['rolling_total_obs'] = gdf_NASA['obs'].rolling(2).sum()

### B.4. Merge NASA dataset again with "first merged" dataset based on the Station and Date

In [ ]:
merged_df = pd.merge(gdf_FEMA_plus_station_id, gdf_NASA, left_on=['stns', 'dateofloss'], right_on=['stns', 'new_date_column'])

In [ ]:
merged_df.keys()

## C. Combined NASA and FEMA Dataset

In [ ]:
merged_df.head(50)

In [ ]:
merged_clean_df=merged_df.drop(columns=["time"])
FEMA_Clean_df.head(10)

In [ ]:
# a=gdf_NASA[gdf_NASA['stns']==179]
# a.new_date_column

In [ ]:
# inds=a['new_date_column']==datetime.datetime(2015,2,21)
# a[inds]
# # nearest_date(datetime.datetime(2015,2,21),179,gdf_NASA)

In [ ]:
# nasa_station_geometry_df = gdf_NASA[['stns', 'geometry']].drop_duplicates()

In [ ]:
# nasa_station_geometry_df

In [ ]:
# nasa_geometry_df = gdf_NASA[['geometry']].drop_duplicates()

In [ ]:
# nasa_geometry_df

In [ ]:
# nasa_geometry_df_groupby = gdf_NASA.groupby(['stn_lat', 'stn_lon']).agg({'new_date_column': ['min', 'max', 'count']})
# #[['new_date_column']].min()

In [ ]:
# nasa_geometry_df_groupby

In [ ]:
# nasa_geometry_df_groupby.to_csv('nasa_geometry_df_groupby_lookup.csv')

In [ ]:
# nasa_station_geometry_df.to_csv('nasa_station_geometry_lookup.csv')

In [ ]:
# gdf_NASA.iloc[3973843]
# gdf_NASA.iloc[1988859]

In [ ]:
# gdf_NASA['obs'].replace

In [ ]:
# from datetime import timedelta

# def nearest_date(vec):
#     NASA_DataFrame=gdf_NASA
# #     FEMA_date, NASA_Station, NASA_DataFrame
# #     Start=FEMA_date - timedelta(days=7)
#     Start=vec[0] - timedelta(days=7)
#     End=vec[0] + timedelta(days=1)
#     #greater than the start date and smaller than the end date
#     mask = (NASA_DataFrame["stns"]==vec[1])&(NASA_DataFrame['new_date_column'] == Start) & (NASA_DataFrame['new_date_column'] == End)
#     df = NASA_DataFrame.loc[mask]
#     Last7DyasSumPrecipitation=df['obs'].sum()     
#     return Last7DyasSumPrecipitation


In [ ]:
# gdf_FEMA_plus_station_id['Perc7Days'] = gdf_FEMA_plus_station_id[['dateofloss', 'stns']].apply(nearest_date, axis=1)


In [ ]:
# gdf_FEMA_plus_station_id

In [ ]:
# from datetime import timedelta

# def nearest_date(vec):
#     NASA_DataFrame=gdf_NASA
# #     FEMA_date, NASA_Station, NASA_DataFrame
# #     Start=FEMA_date - timedelta(days=7)
#     Start=vec[0] - timedelta(days=7)
#     End=vec[0] + timedelta(days=1)
#     #greater than the start date and smaller than the end date
#     mask = (NASA_DataFrame["stns"]==vec[1])&(NASA_DataFrame['new_date_column'] == vec[0])& (NASA_DataFrame['new_date_column'] == End

#     df = NASA_DataFrame.loc[mask]
#     Last7DyasSumPrecipitation=df['obs'].sum()     
#     return Last7DyasSumPrecipitation


In [ ]:
# gdf_FEMA_plus_station_id1=ckdnearest(gdf_FEMA[:100], gdf_NASA[:100]) #gdf_NASA[:10000])


In [ ]:
# gdf_FEMA_plus_station_id1['Perc7Days'] = gdf_FEMA_plus_station_id1[['dateofloss', 'stns']].apply(nearest_date, axis=1)
